#### 개요

* 전체 데이터를 대상으로 공통 컬럼 확인

#### 라이브러리 불러오기

In [1]:
import os
import sys
import numpy as np
import pandas as pd

# 사용자정의 함수: key를 입력하면 해당 데이터를 데이터프레임으로 읽어옴
from aiddd.data_manager import read_data, write_data, _PATH_READ_FILES, _PATH_READ_WRITE_FILES
from aiddd.utils import show_commas

#### 모든 데이터 불러오기

In [2]:
df = {
    'kbt': {
        '고객': read_data('kbt 고객'),
        '인입선': read_data('kbt 인입선'),
        '장주애자': read_data('kbt 장주애자'),
        '전선': read_data('kbt 전선'),
        '전주': read_data('kbt 전주'),
        '접지': read_data('kbt 접지'),
        '지지선': read_data('kbt 지지선'),
    },
    '공사비': {
        '공사비': read_data('공사비'),
    },
    '기별': {
        '애자': read_data('기별 애자'),
        '인입선': read_data('기별 인입선'),
        '장주': read_data('기별 장주'),
        '전선': read_data('기별 전선'),
        '전주': read_data('기별 전주'),
        '접지': read_data('기별 접지'),
        '지지선': read_data('기별 지지선'),
    },
    '설계': {
        '고객': read_data('설계 고객'),
        '애자': read_data('설계 애자'),
        '인입선': read_data('설계 인입선'),
        '장주': read_data('설계 장주'),
        '전선': read_data('설계 전선'),
        '전주': read_data('설계 전주'),
        '접지': read_data('설계 접지'),
        '지지선': read_data('설계 지지선'),
    },
}

#### 각 데이터셋별 공통 컬럼명 추출

In [3]:
# 각 데이터셋별 공통 컬럼명 추출
for key in df.keys():
    print(f'{key}:')
    common_cols = df[key][list(df[key].keys())[0]].columns
    if len(df[key].keys()) != 1:
        for key2 in list(df[key].keys())[1:]:
            common_cols = [
                col for col in common_cols if col in df[key][key2].columns
            ]
    print(common_cols)
        

kbt:
['CMRM_CD', 'GID', 'FAC_STAT_CD', 'CSNTI_DT', 'LST_RVSE_DT', 'DESN_OBJ_GID']
공사비:
Index(['공사번호', '공사변경순번', '배전공사형태코드', '회사분공사비', '도급분공사비', '자본적금액', '수익적금액',
       '본사분재료비', '외자분재료비', '내자분재료비',
       ...
       '최초등록일시', '최초등록자사번', '최종변경일시', '최종변경자사번', '사업소명', '접수종류코드', '접수종류명',
       '공사형태코드', '공사형태명', '계약전력'],
      dtype='object', length=143)
기별:
['공사단위', '공사번호', '공사변경순번', '전산화번호', '설계설비GID', '배전공사형태코드', '기단위공통순번', '자재번호', '기단위자재수량', '자재단가', '자재연관전개번호', '품번호', '공종코드', '공종규격코드1', '공종규격코드2', '기단위품수량', '기단위동시수량', '기단위전환수량', '직접노무비', '공구손료', '기계경비', '잡재료비', '품연관전개번호', '해설순번', '해설번호', '해설코드1', '해설코드2', '기단위해설값', '해설명', '해설구분코드', '기단위작업계수', '사업소명', '접수종류코드', '접수종류명', '공사형태코드', '공사형태명', '계약전력']
설계:
['공사번호', '공사변경순번', '전산화번호', '공사구분코드', 'GISID', '최초등록일시', '최초등록자사번', '최종변경일시', '최종변경자사번', '사업소명', '접수종류코드', '접수종류명', '공사형태코드', '공사형태명', '계약전력']


#### 데이터셋 간 공통 컬럼명 추출

In [4]:
def get_common_cols(*cols):
    common_cols = df[cols[0]][list(df[cols[0]].keys())[0]].columns
    for key in cols:
        for key2 in list(df[key].keys()):
            common_cols = [
                col for col in common_cols if col in df[key][key2].columns
            ]
    
    return common_cols

In [5]:
get_common_cols('공사비', '기별')

['공사번호',
 '공사변경순번',
 '배전공사형태코드',
 '잡재료비',
 '직접노무비',
 '기계경비',
 '공구손료',
 '사업소명',
 '접수종류코드',
 '접수종류명',
 '공사형태코드',
 '공사형태명',
 '계약전력']

In [6]:
get_common_cols('공사비', '설계')

['공사번호',
 '공사변경순번',
 '최초등록일시',
 '최초등록자사번',
 '최종변경일시',
 '최종변경자사번',
 '사업소명',
 '접수종류코드',
 '접수종류명',
 '공사형태코드',
 '공사형태명',
 '계약전력']

In [7]:
get_common_cols('설계', '기별')

['공사번호',
 '공사변경순번',
 '전산화번호',
 '사업소명',
 '접수종류코드',
 '접수종류명',
 '공사형태코드',
 '공사형태명',
 '계약전력']

In [8]:
get_common_cols('공사비', '설계', '기별')

['공사번호', '공사변경순번', '사업소명', '접수종류코드', '접수종류명', '공사형태코드', '공사형태명', '계약전력']

#### 공통 컬럼별 유니크값 확인

In [9]:
def get_common_cols_nunique(*cols):
    common_cols = get_common_cols(*cols)
    df_nunique = pd.DataFrame()
    
    for key1 in cols:
        for key2 in df[key1].keys():
            nunique = df[key1][key2][common_cols].nunique()
            nunique.name = f'{key1}-{key2}'
            df_nunique = pd.concat([df_nunique, nunique], axis=1)
    return df_nunique.T

In [10]:
get_common_cols_nunique('공사비', '설계', '기별')

,공사번호,공사변경순번,사업소명,접수종류코드,접수종류명,공사형태코드,공사형태명,계약전력
공사비-공사비,7097,1,1,14,14,3,3,261
설계-고객,3681,1,1,9,9,3,3,114
설계-애자,3941,1,1,12,12,3,3,184
설계-인입선,5181,1,1,11,11,3,3,172
설계-장주,3942,1,1,12,12,3,3,184
설계-전선,4289,1,1,13,13,3,3,177
설계-전주,3886,1,1,13,13,3,3,195
설계-접지,4769,1,1,13,13,3,3,223
설계-지지선,3502,1,1,12,12,3,3,186
기별-애자,3851,1,1,12,12,3,3,183


In [11]:
df['공사비']['공사비']['공사변경순번'].value_counts()

공사변경순번
99    7097
Name: count, dtype: int64

In [12]:
df['공사비']['공사비']['사업소명'].value_counts()

사업소명
동청주지사    7097
Name: count, dtype: int64

In [13]:
df['공사비']['공사비']['접수종류명'].value_counts()

접수종류명
신설(상용/임시)     5810
신설(발전기)        597
증설             185
공급방식변경증설       155
계약종별변경증설       127
단순공급방식변경        74
해지후재사용(신규)      46
공급지점변경          43
일부해지(신규)        27
계약단위합병          23
신증설(예비전력)        6
증설(발전기)          2
계약변경(발전기)        1
계약단위분할           1
Name: count, dtype: int64

In [14]:
df['공사비']['공사비']['공사형태명'].value_counts()

공사형태명
외선소요         7031
외선소요(선공급)      55
외선불요           11
Name: count, dtype: int64

In [15]:
df['kbt']['고객'].head()

,CMRM_CD,GID,CNTR_NO,FAC_STAT_CD,OLD_NDIS_CNTR_NO,PH_CD,ERRO_CONT,CONS_NO_SEQNO,CSNTI_DT,LST_RVSE_DT,LV_LIWI_GID,LVCA_GID,DESN_OBJ_GID
0,CB,393990,1309504595,JS,NaN,NaN,,NaN,NaN,NaN,356071379,NaN,NaN
1,CB,767581,1303949181,EI,144510324151104****,NaN,NaN,NaN,NaN,NaN,285658488,NaN,NaN
2,CB,838949,1304281201,EI,204510344007804****,NaN,NaN,NaN,NaN,NaN,354723067,NaN,NaN
3,CB,858921,1306016743,RZ,034510402004400****,NaN,NaN,NaN,NaN,NaN,307875520,NaN,NaN
4,CB,1185858,1304256532,EI,204510341100446****,NaN,NaN,NaN,NaN,NaN,363761956,NaN,NaN


In [16]:
df['공사비']['공사비'].head()

,공사번호,공사변경순번,배전공사형태코드,회사분공사비,도급분공사비,자본적금액,수익적금액,본사분재료비,외자분재료비,내자분재료비,...,최초등록일시,최초등록자사번,최종변경일시,최종변경자사번,사업소명,접수종류코드,접수종류명,공사형태코드,공사형태명,계약전력
0,442720154691,99,,23052610,15962515,35272725,3742400,23052610,0,23052610,...,20/01/10,MDE1707278,20/01/10,MDE1707278,동청주지사,11,신설(상용/임시),2,외선소요,7700
1,442720154711,99,,974130,1390620,2364750,0,974130,0,974130,...,19/11/09,ETL_DEV,19/11/09,ETL_DEV,동청주지사,11,신설(상용/임시),2,외선소요,5
2,442720154728,99,,67000695,38982058,102787081,3103395,67000695,0,67000695,...,19/11/09,ETL_DEV,19/11/09,ETL_DEV,동청주지사,11,신설(상용/임시),2,외선소요,1450
3,442720155639,99,,248504,1162131,1410635,0,248504,0,248504,...,20/03/05,2007200722,20/03/05,2007200722,동청주지사,11,신설(상용/임시),2,외선소요,3
4,442720163475,99,,12995746,8315348,21311094,0,12995746,0,12995746,...,19/11/09,ETL_DEV,19/11/09,ETL_DEV,동청주지사,11,신설(상용/임시),2,외선소요,35


In [17]:
df['kbt']['전주'].head()

,CMRM_CD,GID,TS2_OFFICE_CD,NAT_ZONE_NO,MESH_NO,LINE_NO,POLE_FORM_CD,POLE_KND_CD,POLE_SPEC_CD,PNTPL_YN,...,TEXT_GIS_SPC_MATT_ANNXN,TEXT_GIS_ACCU_LOC_ADDR_ANNXN,UPPO_FAC_GID,SYMB_SEQNO,LWER_FAC_DESC,NAT_BROFC_NO_DSBD_YN,HINST_POLE_USE_YN,SD_PLCLMB_MNPW_RSN_CD,SD_NOPLT_STS_CD,ESTM_PRDT_MTTR_GID
0,CB,396722692,4510,22,1479F992,351R17,O,C,10.0,Z,...,NaN,NaN,NaN,0,NaN,NaN,Y,NaN,NaN,NaN
1,CB,352979780,4510,22,1479G741,351D7,O,H,16.0,C,...,NaN,NaN,NaN,0,NaN,NaN,N,NaN,NaN,NaN
2,CB,352979784,4510,22,1479G742,351D8,O,C,10.0,Z,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,CB,352193945,4510,22,1479G851,351D5,O,H,16.0,C,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,CB,361872909,4510,22,1479G852,351D5,G,C,12.0,Z,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


#### 통계

In [18]:
def get_common_cols_nunique(*cols):
    common_cols = get_common_cols(*cols)
    df_nunique = pd.DataFrame()
    
    for key1 in cols:
        for key2 in df[key1].keys():
            nunique = df[key1][key2][common_cols].nunique()
            nunique.name = f'{key1}-{key2}'
            df_nunique = pd.concat([df_nunique, nunique], axis=1)
    return df_nunique.T

In [19]:
df_cc = df['공사비']['공사비']
def get_statistics(key1, key2):
    df_data
    f_size = os.path.getsize(_PATH_READ_FILES[f'{key1} {key2}'])
    shape_rows, shape_cols = df[key1][key2].shape[0], df[key1][key2].shape[1]
    

In [20]:
indexs = [
    '파일크기', '레코드수', '컬럼수', 
    '공사번호 수', '일치한 공사번호 수', '일치한 공사번호 합'
]
def get_series(*v):
    values = [val for val in v]
    return pd.Series(values, index=indexs)

In [21]:
df_statistics = pd.DataFrame()

df_cc = df['공사비']['공사비']
f_size = os.path.getsize(_PATH_READ_FILES[f'공사비 공사비'])
shape_rows, shape_cols = df_cc.shape[0], df_cc.shape[1]
unique_rows = df_cc['공사번호'].unique().shape[0]
series = get_series(f_size, shape_rows, shape_cols, unique_rows, 0, 0)
series.name = '공사비'
df_statistics = pd.concat([df_statistics, series], axis=1)

for key1 in ['기별', '설계']:
    for key2 in df[key1].keys():
        df_data = df[key1][key2]
        
        f_size = os.path.getsize(_PATH_READ_FILES[f'{key1} {key2}'])
        shape_rows, shape_cols = df_data.shape[0], df_data.shape[1]
        unique_rows = df_data['공사번호'].unique()
        match_unique_rows = df_cc[df_cc['공사번호'].isin(unique_rows)].shape[0]
        match_rows = df_data[df_data['공사번호'].isin(df_cc['공사번호'])].shape[0]
        series = get_series(
            f_size, shape_rows, shape_cols, 
            unique_rows.shape[0], match_unique_rows, match_rows
        )
        series.name = f'{key1}-{key2}'
        df_statistics = pd.concat([df_statistics, series], axis=1)
        
show_commas(df_statistics.T)
        

,파일크기,레코드수,컬럼수,공사번호 수,일치한 공사번호 수,일치한 공사번호 합
공사비,"4,078,963","7,097",143,"7,097",0,0
기별-애자,"8,609,962","62,753",37,"3,851","3,851","62,753"
기별-인입선,"10,468,734","75,723",37,"8,288","4,862","45,632"
기별-장주,"6,455,652","45,133",37,"3,878","3,878","45,133"
기별-전선,"782,184","5,391",37,"1,242","1,242","5,391"
기별-전주,"11,323,172","83,013",37,"3,749","3,749","83,013"
기별-접지,"3,324,108","23,568",37,"3,375","3,375","23,568"
기별-지지선,"6,212,481","45,452",37,"3,157","3,157","45,452"
설계-고객,"425,479","4,822",17,"3,681","3,681","4,822"
설계-애자,"2,643,953","10,920",71,"3,941","3,941","10,920"


In [22]:
df_statistics = df_statistics.T
df_statistics.index.name = '파일명'
write_data('통계 전체', df_statistics, True)

In [23]:
df_data = read_data('통계 전체').set_index('파일명')
show_commas(df_data)

,파일크기,레코드수,컬럼수,공사번호 수,일치한 공사번호 수,일치한 공사번호 합
파일명,,,,,,
공사비,"4,078,963","7,097",143,"7,097",0,0
기별-애자,"8,609,962","62,753",37,"3,851","3,851","62,753"
기별-인입선,"10,468,734","75,723",37,"8,288","4,862","45,632"
기별-장주,"6,455,652","45,133",37,"3,878","3,878","45,133"
기별-전선,"782,184","5,391",37,"1,242","1,242","5,391"
기별-전주,"11,323,172","83,013",37,"3,749","3,749","83,013"
기별-접지,"3,324,108","23,568",37,"3,375","3,375","23,568"
기별-지지선,"6,212,481","45,452",37,"3,157","3,157","45,452"
설계-고객,"425,479","4,822",17,"3,681","3,681","4,822"


In [24]:
df_statistics = pd.DataFrame()

df_cc = read_data('전처리 공사비')
f_size = os.path.getsize(_PATH_READ_WRITE_FILES['전처리 공사비'])
shape_rows, shape_cols = df_cc.shape[0], df_cc.shape[1]
unique_rows = df_cc['공사번호'].unique().shape[0]
series = get_series(f_size, shape_rows, shape_cols, unique_rows, 0, 0)
series.name = '공사비'
df_statistics = pd.concat([df_statistics, series], axis=1)

for key1 in ['기별', '설계']:
    for key2 in df[key1].keys():
        df_data = df[key1][key2]
        
        f_size = os.path.getsize(_PATH_READ_FILES[f'{key1} {key2}'])
        shape_rows, shape_cols = df_data.shape[0], df_data.shape[1]
        unique_rows = df_data['공사번호'].unique()
        match_unique_rows = df_cc[df_cc['공사번호'].isin(unique_rows)].shape[0]
        match_rows = df_data[df_data['공사번호'].isin(df_cc['공사번호'])].shape[0]
        series = get_series(
            f_size, shape_rows, shape_cols, 
            unique_rows.shape[0], match_unique_rows, match_rows
        )
        series.name = f'{key1}-{key2}'
        df_statistics = pd.concat([df_statistics, series], axis=1)
        
show_commas(df_statistics.T)

,파일크기,레코드수,컬럼수,공사번호 수,일치한 공사번호 수,일치한 공사번호 합
공사비,"436,526","5,810",9,"5,810",0,0
기별-애자,"8,609,962","62,753",37,"3,851","3,322","55,087"
기별-인입선,"10,468,734","75,723",37,"8,288","4,196","29,823"
기별-장주,"6,455,652","45,133",37,"3,878","3,339","39,282"
기별-전선,"782,184","5,391",37,"1,242",790,"2,871"
기별-전주,"11,323,172","83,013",37,"3,749","3,245","67,392"
기별-접지,"3,324,108","23,568",37,"3,375","2,866","17,321"
기별-지지선,"6,212,481","45,452",37,"3,157","2,756","37,313"
설계-고객,"425,479","4,822",17,"3,681","3,531","4,528"
설계-애자,"2,643,953","10,920",71,"3,941","3,393","9,443"


In [25]:
df_statistics = df_statistics.T
df_statistics.index.name = '파일명'
write_data('통계 1차전처리', df_statistics, True)